In [33]:
pip install tensorflow2

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow2 (from versions: none)
ERROR: No matching distribution found for tensorflow2


In [1]:
import numpy as np
import sys
from sklearn.preprocessing import StandardScaler
#np.set_printoptions(threshold=sys.maxsize)
from keras.models import Sequential

from keras.layers import LSTM, Dense, Dropout, BatchNormalization, Embedding
from keras.layers import Conv1D, MaxPooling1D, Activation, Flatten
from keras.preprocessing import sequence
import scipy.io as scio
# fix random seed for reproducibility
np.random.seed(7)
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import pandas as pd

print(os.getcwd())
files = os.listdir()
set = {}
for file in files:


    if file == 'csv':

        new_path = os.path.join(os.getcwd(),file)


        for dataset in os.listdir(new_path):

            head,sep,tail = dataset.partition('.')
            #print(head)
            data = pd.read_csv(os.path.join(new_path,dataset))
            set[head] = data








D:\PPG_Divya


In [ ]:
set['s10_run']

In [3]:

alllabels = pd.DataFrame()
dset = pd.DataFrame()
i=0
for key,value in set.items():
    #subs.append(i.split('_')
    subs=[]
    labels = []
    featureset=[]
    data = pd.DataFrame(value.iloc[:,:9])
    sub_label = key.split('_')
    sub = sub_label[0]
    label = sub_label[1]
    for i in range(len(data)):
        subs.append([sub])
        labels.append(label)
    labels = pd.DataFrame(labels,columns=['Label'])
    subs = pd.DataFrame(subs,columns=['SubjectID'])
    featureset = pd.concat([subs,data,labels],axis=1)
    if i<1:
        dset = featureset
        #alllabels = labels
    else:
        
        f = featureset.copy()
        #l = labels.copy()
        dset = pd.concat([dset,f],axis=0)
        #alllabels = pd.concat([alllabels,l],axis=0)
    i+=1
    

In [5]:
dset

,SubjectID,ecg,peaks,pleth_1,pleth_2,pleth_3,pleth_4,pleth_5,pleth_6,Label
0,s10,33664,0,65589,69333,3164,90553,103093,5652,run
1,s10,33866,0,65589,69333,3164,90558,103077,5647,run
2,s10,34155,0,65595,69334,3181,90544,103098,5660,run
3,s10,34366,0,65591,69349,3175,90544,103098,5660,run
4,s10,34538,0,65583,69343,3186,90554,103093,5651,run
...,...,...,...,...,...,...,...,...,...,...
243729,s9,22079,0,67854,81189,4279,64547,92187,3117,walk
243730,s9,22481,0,67869,81196,4287,64558,92186,3117,walk
243731,s9,23013,0,67866,81176,4282,64558,92186,3117,walk
243732,s9,23631,0,67851,81185,4274,64558,92190,3121,walk


In [6]:
features = dset.drop(['SubjectID','Label'],axis =1)
featureshape = features.shape[1]

In [4]:
dset=dset.drop(['time'],axis=1)

In [9]:
scale = StandardScaler()
fet = scale.fit_transform(features)
dsett= dset.copy()
dsett.iloc[:,1:-1] = fet

In [12]:
dsett

,SubjectID,ecg,peaks,pleth_1,pleth_2,pleth_3,pleth_4,pleth_5,pleth_6,Label
0,1,0.172065,-0.05357,-1.656461,-0.928881,-0.995633,0.976600,1.527440,0.416200,0
1,1,0.206277,-0.05357,-1.656461,-0.928881,-0.995633,0.977192,1.525093,0.411806,0
2,1,0.255225,-0.05357,-1.655510,-0.928743,-0.982392,0.975534,1.528173,0.423230,0
3,1,0.290963,-0.05357,-1.656144,-0.926663,-0.987065,0.975534,1.528173,0.423230,0
4,1,0.320094,-0.05357,-1.657412,-0.927495,-0.978498,0.976718,1.527440,0.415321,0
...,...,...,...,...,...,...,...,...,...,...
243729,21,-1.790092,-0.05357,-1.297314,0.714521,-0.127190,-2.103840,-0.071868,-1.811394,2
243730,21,-1.722005,-0.05357,-1.294936,0.715492,-0.120959,-2.102537,-0.072014,-1.811394,2
243731,21,-1.631900,-0.05357,-1.295411,0.712719,-0.124853,-2.102537,-0.072014,-1.811394,2
243732,21,-1.527229,-0.05357,-1.297790,0.713967,-0.131084,-2.102537,-0.071428,-1.807879,2


In [13]:
grouped = dset.groupby('SubjectID')

In [11]:
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
le = LabelEncoder()
enc = OrdinalEncoder()
dsett['Label']=le.fit_transform(dsett['Label'])
dsett['SubjectID'] = le.fit_transform(dsett['SubjectID'])

In [14]:
from keras.layers import Dropout
model = Sequential()
#model.add(Embedding(tot_length, embedding_vector_length, input_length=max_length))
#model.add(Dropout(0.2))
model.add(LSTM(50,activation = 'tanh',return_sequences=True, input_shape = (1,featureshape)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(80,activation = 'sigmoid',return_sequences=True, stateful=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.build()
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 50)             11800     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 batch_normalization (BatchN  (None, 1, 50)            200       
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 1, 80)             41920     
                                                                 
 dropout_1 (Dropout)         (None, 1, 80)             0         
                                                                 
 dense (Dense)               (None, 1, 1)              81        
                                                        

In [15]:
for group in grouped:
    
    batch = pd.DataFrame(columns = dsett.columns)
    for subjects in group:
        
        filter=dsett['SubjectID']==subjects
        for key,item in dsett[filter].dropna().iterrows():
            
            batch = batch.append(item)
    X= batch[features.columns]
    X= np.resize(X,(X.shape[0],1,X.shape[1]))
    y = batch['label']
    model.fit(X,y,batch_size=len(X),epochs = 10)

MemoryError: Unable to allocate 87.9 TiB for an array with shape (16217132, 745202) and data type float64

In [ ]:
lab

In [ ]:
dsett.columns